In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_burden = pd.read_csv('data_2020.csv')
data_burden = data_burden[data_burden['burden'].notna()]
data_burden = data_burden[data_burden.columns[data_burden.isnull().mean() < 0.2]]

data_burden.columns[data_burden.isnull().any()]

for column in data_burden.columns:
    data_burden[column].fillna(data_burden[column].mode()[0], inplace=True)

median = data_burden.loc[data_burden['agecr'] < 200, 'agecr'].median()
data_burden["agecr"] = np.where(data_burden["agecr"] > 200, median,data_burden['agecr'])

# random forest
from sklearn.ensemble import RandomForestRegressor

X_feature = data_burden.drop('burden', axis = 1)
y_feature = data_burden['burden']

#m = sqrt(p)+1 features
nfeatures = data_burden.shape[1] - 1
feature_model = RandomForestRegressor(max_features = int(np.sqrt(nfeatures))+1, random_state = 1) #random_state ensure random bagging
feature_model.fit(X_feature,y_feature)


df_feature = pd.DataFrame(zip(X_feature.columns, feature_model.feature_importances_), columns = ['feature','importance'])
df_feature = df_feature.sort_values(by=['importance'], ascending=False)

df_feature[0:29]

In [ ]:
sns.barplot(data = df_feature[0:19], x = 'importance', y='feature', orient = 'h');
plt.title('Feature Importance Plot Random Forest')
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')

In [ ]:
#select the top few of the high important features while avoiding multi-colinearity
data_burden = data_burden[['year', 'Q18', 'HOURS', 'adls', 'q22a', 'q22b', 'q22d', 'N3', 'q22c', 'iadls', 'q22g', 'q22f', 'q23d', 'banlives', 'q23c', 'q22e', 'burden']]

In [ ]:
#See the tally of data points of each of the illnesses
data_burden.groupby(['Q18'])['Q18'].count().sort_values(ascending=False)

In [ ]:
#Select the only the ones with sufficient datapoints --- 80+ was arbitrarily chosen 
lst = [24.0, 3.0, 23.0, 32.0, 14.0, 22.0, 19.0, 30.0, 16.0, 8.0, 18.0, 5.0, 20.0, 13.0, 43.0]
data_burden = data_burden.loc[data_burden['Q18'].isin(lst)]
data_burden["Q18"] = data_burden["Q18"].astype(str)

#re-code the numbers to actual illness
data_burden["Q18"] = data_burden["Q18"].replace(["3.0","5.0","8.0","14.0","16.0","18.0","19.0","20.0","22.0","23.0","24.0","30.0","32.0"
                           , "13.0", '43.0'], ["Alzheimer","Arthritis", "BackProblems"
                           ,"Cancer", "Diabetes", "Falling", "HeartDisease",
                            "LungDisease", "MentalIllness", "MobilityProblem", "Aging",  "Stroke",
                            "Surgery", "BrokenBones", "Alzheimer"])
data_burden.head()

In [ ]:
#Change the data types to categorical
lst2 = ['q22a', 'q22b', 'q22d', 'N3', 'q22c', 'q22g', 'q22f', 'q23d', 'banlives', 'q23c', 'q22e']
data_burden[lst2] = data_burden[lst2].astype(object)

In [ ]:
#remove useless values such as "don't know", "not answered"

values = [3.0, 4.0]
data_burden = data_burden[
    (data_burden.q22a.isin(values) == False) \
        & (data_burden.q22b.isin(values) == False) \
        & (data_burden.q22d.isin(values) == False)
        & (data_burden.N3.isin(values) == False) \
        & (data_burden.q22c.isin(values) == False) \
        & (data_burden.q22g.isin(values) == False) \
        & (data_burden.q22f.isin(values) == False) \
        & (data_burden.q23d.isin(values) == False) \
        & (data_burden.q23c.isin(values) == False) \
        & (data_burden.q22e.isin(values) == False)                        
                        ]

data_burden = data_burden[data_burden['banlives'] != 3.0]

In [ ]:
data_burden.head()

In [ ]:
#re-coding survey numbers to actual response
data_burden["q22a"] = data_burden["q22a"].astype(str)
data_burden["q22a"] = data_burden["q22a"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q22b"] = data_burden["q22b"].astype(str)
data_burden["q22b"] = data_burden["q22b"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q22d"] = data_burden["q22d"].astype(str)
data_burden["q22d"] = data_burden["q22d"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["N3"] = data_burden["N3"].astype(str)
data_burden["N3"] = data_burden["N3"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q22c"] = data_burden["q22c"].astype(str)
data_burden["q22c"] = data_burden["q22c"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q22g"] = data_burden["q22g"].astype(str)
data_burden["q22g"] = data_burden["q22g"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q22f"] = data_burden["q22f"].astype(str)
data_burden["q22f"] = data_burden["q22f"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q23d"] = data_burden["q23d"].astype(str)
data_burden["q23d"] = data_burden["q23d"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q23c"] = data_burden["q23c"].astype(str)
data_burden["q23c"] = data_burden["q23c"].replace(["1.0","2.0"], ["Yes","No"])
data_burden["q22e"] = data_burden["q22e"].astype(str)
data_burden["q22e"] = data_burden["q22e"].replace(["1.0","2.0"], ["Yes","No"])

data_burden["banlives"] = data_burden["banlives"].astype(str)
data_burden["banlives"] = data_burden["banlives"].replace(["1.0","2.0"], ["Yes","No"])

In [ ]:
#renaming the column names to laymens terms
data_burden.columns = ['year', 'illness', 'hours', 'adls', 'help_with_bed', 'help_with_dressed', 'help_with_bathe', 'help_with_med', 'help_with_toilet', 'iadls', 'giving_medicine', 'help_with_feeding', 'preparing_meals', 'live_with_cr', 'help_housework', 'help_with_diapers', 'burden']
data_burden = data_burden.reset_index(drop=True)

## Plots

In [ ]:
"""
df_1 = data_burden.groupby(['illness'])['burden'].median()
df_1 = df_1.to_frame()
df_1.reset_index(inplace=True)
df_1 = df_1.rename(columns = {'index':'illness'})
df_1['illness'] = df_1['illness'].astype(object)

fig = plt.figure(1, [20, 8])
fig.clf()

ax = fig.add_subplot(111)
ax.set_xlim(-1,14)
plt.setp(ax.get_xticklabels(), fontsize=10, rotation='vertical')
plt.bar(df_1['illness'],df_1['burden'])

plt.axhline(y=(df_1[df_1['illness'] == 'Aging']['burden'][0]),linewidth= 3, color='r', linestyle= 'dotted')
plt.title(label = "burden", fontsize=40)
plt.plot()
plt.show()
"""

In [ ]:
"""
sns.catplot(x="burden", y="help_with_diapers", kind="box", data=data_2014)
sns.catplot(x="help_with_diapers", y="burden", kind="box", data=data_2014)
"""

In [ ]:
data_burden.head()

In [ ]:
#sns.catplot(x="burden", y="hours", kind="box", data=data_burden)
sns.catplot(x="burden", y="hours", kind="violin", data=data_burden)

#sns.catplot(x="burden", y="iadls", kind="box", data=data_burden)
sns.catplot(x="burden", y="iadls", kind="violin", data=data_burden)

#sns.catplot(x="burden", y="adls", kind="box", data=data_burden)
sns.catplot(x="burden", y="adls", kind="violin", data=data_burden)

#sns.catplot(x="help_with_bed", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_with_bed", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="help_with_dressed", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_with_dressed", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="help_with_bathe", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_with_bathe", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="help_with_med", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_with_med", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="help_with_toilet", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_with_toilet", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="giving_medicine", y="burden", kind="box", data=data_burden)
sns.catplot(x="giving_medicine", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])
#sns.boxplot(x='species', y='sepal_length', data=df, order=["versicolor", "virginica", "setosa"])

#sns.catplot(x="help_with_feeding", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_with_feeding", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="preparing_meals", y="burden", kind="box", data=data_burden)
sns.catplot(x="preparing_meals", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="live_with_cr", y="burden", kind="box", data=data_burden)
sns.catplot(x="live_with_cr", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="help_housework", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_housework", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])

#sns.catplot(x="help_with_diapers", y="burden", kind="box", data=data_burden)
sns.catplot(x="help_with_diapers", y="burden", kind="violin", data=data_burden, order=["Yes", "No"])



sns.catplot(x="burden", y="illness",
            kind="violin", data=data_burden)


In [ ]:
data_2015 = data_burden[data_burden['year'] == 2014]
data_2020 = data_burden[data_burden['year'] == 2019]

In [ ]:
data_2015.head()

In [ ]:
data_2015.groupby(['illness'])['burden'].describe()

In [ ]:
data_2020.groupby(['illness'])['burden'].describe()

In [ ]:
sns.catplot(x="burden", y="illness",
            kind="violin", data=data_2015)

sns.catplot(x="burden", y="illness",
            kind="violin", data=data_2020, order=["Aging", "MobilityProblem", "Arthritis", "Cancer", "Diabetes", "MentalIllness", "Alzheimer", "Surgery", "HeartDisease", "Stroke", "BackProblems", "BrokenBones", "LungDisease", "Falling"])

### Narrow Down Features

In [ ]:
#Dropping features with small correlations
data_burden2 = data_burden.drop(['adls', 'help_housework', 'live_with_cr', 'preparing_meals', 'giving_medicine', 'help_with_med'], axis=1)

In [ ]:
#Splitting the continuous variables and the categorical variables 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df_cont2 = data_burden2.select_dtypes(include=numerics)
df_cat2 = data_burden2.select_dtypes(include = 'object')

#dummy coding the categorical variables
df_cat_dc2 = pd.get_dummies(df_cat2)
df_reg2 = pd.concat([df_cont2, df_cat_dc2], axis = 1)

In [ ]:
#Helper function for 10-CV
# 10-Fold Cross Validation
def cross_validation (df, func):
    from sklearn.model_selection import KFold
    kf = KFold(n_splits = 10, shuffle = True, random_state = 10)
    kf_rmse = []
    
    for train, test in kf.split(df):
        X_train = df.iloc[train].loc[:, df.columns != 'burden']
        X_train = X_train.squeeze()
        X_test = df.iloc[test].loc[:, df.columns != 'burden']
        y_train = df.iloc[train].loc[:,'burden']
        y_test = df.iloc[test].loc[:,'burden']
        
        reg = func.fit(X_train, y_train)
        y_hat = reg.predict(X_test)
        
        from sklearn.metrics import mean_squared_error
        kf_rmse.append(mean_squared_error(y_test, y_hat, squared=False))
        
    kf_RMSE = (1/10) * np.sum(kf_rmse)
        
    return (kf_RMSE)

In [ ]:
#Use 2014 data as train and 2020 data as test
X_train2 = df_reg2[df_reg2['year'] == 2014]
y_train2 = X_train2['burden']
X_train2 = X_train2.drop('burden', axis = 1)
df_train2 = pd.concat([X_train2, y_train2], axis=1, join='outer')

X_test2 = df_reg2[df_reg2['year'] == 2019]
y_test2 = X_test2['burden']
X_test2 = X_test2.drop('burden', axis = 1)

## Linear Regression with Narrowed Down Features

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.linear_model import LinearRegression
model1 = LinearRegression()

model1.fit(X_train2, y_train2)

y_hat11 = model1.predict(X_test2)

score11 = np.mean(cross_val_score(estimator = model1, X = X_train2, y = y_train2, cv = 10))
model11_train_rmse = mean_squared_error(y_test2, y_hat11, squared=False)
model11_cv_rmse = cross_validation(df_reg2, model1)

print('test RMSE = ', model11_train_rmse)
print('10 CV RMSE = ', model11_cv_rmse)
print('score = ', score11)

In [ ]:
plt.figure(figsize = (8,8))
plt.scatter(x = y_test2, y = y_hat11, s = 8, label = "Test")
plt.scatter(x = y_train2, y = model1.predict(X_train2), s = 8, label = "Train")
plt.scatter(x = df_reg2['burden'], y = df_reg2['burden'], s = 8, label = "True")
plt.plot([0,6], [0,6], color = "r")
plt.legend(loc=0)
plt.title("Predicted Value vs True Value for Linear Regression")
plt.xlabel("True Value")
plt.ylabel("Predicted Value")

### Random Forest with Narrowed Down Features

In [ ]:
randomForest = RandomForestRegressor(random_state = 0)
grid_para_forest = {'n_estimators': [100,500,1000,2500,5000],
'max_depth': [10,15,20,30,40,50],
'max_features' : [5,7,15]}
from sklearn.model_selection import GridSearchCV

grid_search_forest = GridSearchCV(randomForest, grid_para_forest, cv=10, n_jobs = 5, verbose=1)
grid_search_forest.fit(X_train2, y_train2)

model2_2 = grid_search_forest.best_estimator_
yhat_2_2 = model2_2.predict(X_test2)

In [ ]:
model2_2_train_rmse = mean_squared_error(y_test2, yhat_2_2, squared=False)
model2_2_cv_rmse = cross_validation(df_reg2, model2_2)
score2_2 = np.mean(cross_val_score(estimator = model2_2, X = X_train2, y = y_train2, cv = 10))

print('test RMSE = ', model2_2_train_rmse)
print('10 CV RMSE = ', model2_2_cv_rmse)
print('score = ', score2_2)

In [ ]:
plt.figure(figsize = (8,8))
plt.scatter(x = y_test2, y = yhat_2_2, s = 8, label = "Test")
plt.scatter(x = y_train2, y = model2_2.predict(X_train2), s = 8, label = "Train")
plt.scatter(x = df_reg2['burden'], y = df_reg2['burden'], s = 8, label = "True")
plt.plot([0,6], [0,6], color = "r")
plt.legend(loc=0)
plt.title("Predicted Value vs True Value for Random Forest")
plt.xlabel("True Value")
plt.ylabel("Predicted Value")

In [ ]:
sorted_importance = sorted(zip(df_reg2.drop('burden', axis = 1).columns, model2_2.feature_importances_), key=lambda t:t[1], reverse = True)
a, b = zip(*sorted_importance)
plt.figure(figsize = (10,10))
df = pd.DataFrame({'feature_name':a, 'importance_score':b})
sns.barplot(data = df, x = 'importance_score', y='feature_name', orient = 'h');
plt.title('Feature Importance Plot Random Forest')
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')

#### Run the Random Forest Again with Important Features

In [ ]:
imp_cols = df[df['importance_score'] > 0.004]['feature_name'].tolist()

In [ ]:
grid_search_forest.fit(X_train2[imp_cols], y_train2)

model22_2 = grid_search_forest.best_estimator_
yhat_22_2 = model22_2.predict(X_test2[imp_cols])

In [ ]:
model22_2_train_rmse = mean_squared_error(y_test2, yhat_22_2, squared=False)
model22_2_cv_rmse = cross_validation(df_reg2, model22_2)
score22_2 = np.mean(cross_val_score(estimator = model22_2, X = X_train2, y = y_train2, cv = 10))

print('test RMSE = ', model22_2_train_rmse)
print('10 CV RMSE = ', model22_2_cv_rmse)
print('score = ', score22_2)

In [ ]:
plt.figure(figsize = (8,8))
plt.scatter(x = y_test2, y = yhat_22_2, s = 8, label = "Test")
plt.scatter(x = y_train2, y = model22_2.predict(X_train2), s = 8, label = "Train")
plt.scatter(x = df_reg2['burden'], y = df_reg2['burden'], s = 8, label = "True")
plt.plot([0,6], [0,6], color = "r")
plt.legend(loc=0)
plt.title("Predicted Value vs True Value for Random Forest 2")
plt.xlabel("True Value")
plt.ylabel("Predicted Value")

## Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(random_state = 0)

In [ ]:
grid_para_gb = {'n_estimators': [100,500,1000,2500,5000],
                   'learning_rate':[0.01,0.05,0.1],
                   'max_depth':range(1,6),
                   'max_features' : [5,7,15]}

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
grid_search_gb = GridSearchCV(gbm, grid_para_gb, cv=5, n_jobs = 5, verbose = 1)
grid_search_gb.fit(X_train2, y_train2)


In [ ]:
model3 = grid_search_gb.best_estimator_
yhat_3 = model3.predict(X_test2)

In [ ]:
model3_train_rmse = mean_squared_error(y_test2, yhat_3, squared=False)
model3_cv_rmse = cross_validation(df_reg2, model3)
score3 = np.mean(cross_val_score(estimator = model1, X = X_train2, y = y_train2, cv = 10))

print('train RMSE = ', model3_train_rmse)
print('10 CV RMSE = ', model3_cv_rmse)
print('score = ', score3)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(x = y_test2, y = yhat_3, s = 8, label = "Test")
plt.scatter(x = y_train2, y = model3.predict(X_train2), s = 8, label = "Train")
plt.scatter(x = df_reg2['burden'], y = df_reg2, s = 8, label = "True")
plt.plot([-2,8],[-2,8], color = "r")
plt.legend(loc = 0)
plt.title("Predicted Value vs True Value for Gradient Boost")
plt.xlabel("True Value")
plt.ylabel("Predicted Value")

In [ ]:
sorted_importance = sorted(zip(df_reg2.columns, model3.feature_importances_), key=lambda t:t[1], reverse=True)
a, b = zip(*sorted_importance)
plt.figure(figsize = (10,10))
df = pd.DataFrame({'feature_name':a, 'importance_score':b})
sns.barplot(data = df, x = 'importance_score', y= 'feature_name', orient = 'h');
plt.title('Feature Importance Plot Gradient Boosting')
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')

In [ ]:
imp_cols = df[df['importance_score'] > 0.004]['feature_name'].tolist()

In [ ]:
grid_search_gb.fit(X_train2[imp_cols], y_train2)

model33 = grid_search_gb.best_estimator_
yhat_33 = model33.predict(X_test2[imp_cols])

In [ ]:
model33_train_rmse = mean_squared_error(y_test2, yhat_33, squared=False)
model33_cv_rmse = cross_validation(df_reg2, model33)
score33 = np.mean(cross_val_score(estimator = model33, X = X_train2, y = y_train2, cv = 10))

print('train RMSE = ', model33_train_rmse)
print('10 CV RMSE = ', model33_cv_rmse)
print('score = ', score33)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(x = y_test2, y = yhat_33, s = 8, label = "Test")
plt.scatter(x = y_train2, y = model33.predict(X_train2), s = 8, label = "Train")
plt.scatter(x = df_reg2['burden'], y = df_reg2, s = 8, label = "True")
plt.plot([-2,8],[-2,8], color = "r")
plt.legend(loc = 0)
plt.title("Predicted Value vs True Value for Gradient Boost")
plt.xlabel("True Value")
plt.ylabel("Predicted Value")

### RMSE Summary

In [ ]:
x = ['LinReg', 'RandomForest1', 'RandomForest2', 'GB1', 'GB2']
y1 = [model11_train_rmse, model2_2_train_rmse, model22_2_train_rmse, model3_train_rmse, model33_train_rmse]
y2 = [model11_cv_rmse, model2_2_cv_rmse, model22_2_cv_rmse, model3_cv_rmse, model33_cv_rmse]

plt.plot(x, y2, label = "10-CV RMSE", c='r')
plt.plot(x, y1, label = "Train RMSE", c='b')
plt.legend()
plt.grid()
plt.show()

## Score Summary

In [ ]:
x = ['LinReg', 'RandomForest1', 'RandomForest2', 'GB1', 'GB2']
y1 = [score11, score2_2, score22_2, score3, score33]

plt.plot(x, y1, label = "10-CV Score", c='b')
plt.legend()
plt.grid()
plt.show()